In [50]:
# import everything ml
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import re
import string
from wordcloud import WordCloud
import nltk
from collections import Counter
import contractions
from tqdm import tqdm



In [51]:
true_dataset = pd.read_csv("./data/fakenews/True.csv")
false_dataset = pd.read_csv("./data/fakenews/Fake.csv")

In [52]:
def combine_and_randomize_datasets(true_dataset, false_dataset):
    true_arr = np.ones(len(true_dataset))
    false_arr = np.zeros(len(false_dataset))

    true_dataset['pred'] = true_arr
    false_dataset['pred'] = false_arr
    combined_dataset = pd.concat([true_dataset, false_dataset], ignore_index=True)
    combined_dataset = combined_dataset.sample(frac=1).reset_index(drop=True)

    return combined_dataset

combined = combine_and_randomize_datasets(true_dataset, false_dataset)

In [53]:
combined

,title,text,subject,date,pred
0,Indonesia detains 18 in pre-emptive bid to boo...,JAKARTA (Reuters) - Indonesian anti-terrorism ...,worldnews,"December 11, 2017",1.0
1,John McCain: Russian Election Meddling Is Eve...,Sen. John McCain is weighing in on Russia s me...,News,"February 18, 2017",0.0
2,McConnell says all need to stand against hate ...,WASHINGTON (Reuters) - U.S. Senate Republican ...,politicsNews,"August 16, 2017",1.0
3,At least five killed in Kenya as violence gree...,NAIROBI (Reuters) - At least five people were ...,worldnews,"November 17, 2017",1.0
4,Progressive Champion Elizabeth Warren Will De...,The Democratic Party is poised to have a conve...,News,"July 25, 2016",0.0
...,...,...,...,...,...
44893,Rights group questions Italy's work with Libya...,ROME (Reuters) - Europe s top human rights org...,worldnews,"October 11, 2017",1.0
44894,Senate Democrats united on debt restructuring ...,"SAN JUAN (Reuters) - U.S. Senate Democrats, in...",politicsNews,"January 27, 2016",1.0
44895,Amerika: ‘Tolerant’ University Educators Exile...,"21st Century Wire says Since the late 1960 s, ...",Middle-east,"November 15, 2016",0.0
44896,MELANIA TRUMP IN RARE ONE-ON-ONE INTERVIEW: Wa...,Melania Trump has been taking a relatively low...,politics,"Feb 24, 2016",0.0


In [54]:


def clean_text(text):
    text = contractions.fix(text)                         # Expand contractions
    text = text.encode("ascii", errors="ignore").decode() # Remove non-ASCII chars
    text = text.lower()                                   # Lowercase
    text = re.sub(r"http\S+|www.\S+", "", text)           # Remove URLs
    text = re.sub(r"<.*?>", "", text)                     # Remove HTML tags
    text = re.sub(r"[^a-z\s]", "", text)                  # Keep only letters and spaces
    text = re.sub(r"\b\w{1}\b", "", text)                 # Remove single-letter words
    text = re.sub(r"\s+", " ", text).strip()              # Remove extra spaces
    words = text.split()
    # words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)



In [55]:
words_dict = {}

def process_words_into_dict(text, words_dict):
    for word in text.split():
        if word not in words_dict:
            words_dict[word] = 1
        else:
            words_dict[word] += 1

for text in combined['text']:
    text = clean_text(text)
    process_words_into_dict(text, words_dict)


In [56]:
for count, value in enumerate(words_dict):
    print(value)
    print(words_dict[value])
    if(count == 100):
        break

jakarta
210
reuters
28670
indonesian
223
antiterrorism
99
police
14599
have
83818
detained
1196
people
41158
with
117413
links
1021
to
535930
militant
1232
groups
6696
in
345268
bid
1320
cut
3250
the
1010046
risk
2166
of
440837
attacks
5346
during
16206
christmas
929
and
406325
new
30934
year
17959
world
11324
biggest
2696
muslimmajority
469
country
16911
said
130002
on
189612
monday
9111
nearsimultaneous
3
churches
285
capital
3591
elsewhere
654
eve
329
killed
5642
nearly
3736
ever
5030
since
14348
authorities
3551
stepped
691
up
29242
security
15683
at
74569
tourist
222
spots
125
for
171729
holiday
618
chief
5968
tito
21
karnavian
5
while
16897
there
30582
was
115865
no
28994
evidence
4626
specific
1237
plot
554
detentions
85
were
40486
made
14875
head
4880
off
8748
trouble
792
we
60131
re
11707
doing
5227
preemptive
92
strike
1304
told
23342
reporters
5679
majority
4456
them
21328
previous
2137
incidents
721
who
69931
had
46111
arrested
2784
earlier
5458
he
131954
that
238361
been
4